# 采样时间安排测试


In [ ]:
# import difflib

from datetime import time, datetime, timedelta
# import numpy as np
import pandas as pd
from nptyping import DataFrame
from schedule_manage_module.schedule_manage import SampleScheduleManage
# from occupational_health_module.occupational_health import OccupationalHealthItemInfo
# from occupational_health_module.other_infos import templates_info

In [ ]:
# company_name: str = '福建申远新材料有限公司'
# project_name: str = '23ZDQ0007-10'

# file_path: str = r'./templates/项目信息试验模板2.xlsx'
# point_info_df: DataFrame = pd.read_excel(file_path, sheet_name='定点') # type: ignore
# personnel_info_df: DataFrame = pd.read_excel(file_path, sheet_name='个体') # type: ignore

In [ ]:
# new_project = OccupationalHealthItemInfo(company_name, project_name, point_info_df, personnel_info_df)

In [ ]:
# test_raw_point_df = (
#     new_project
#     .output_deleterious_substance_info_dict
#     ['1']['定点']
# )

# test_raw_point_df.head()

In [ ]:
test_raw_point_df = pd.read_excel('./templates/采样时间安排模板2.xlsx', sheet_name='定点')

test_raw_point_df.head()

In [ ]:
instruments_raw = {
    '收集方式': ['大气', '大气', '大气', '大气', '大气', '大气','粉尘','粉尘', '粉尘','粉尘','粉尘','粉尘','粉尘','粉尘'],
    '代号': ['1Q1', '2Q1', '2Q2', '3Q1', '4Q1', '4Q2', '1F1', '1F2', '2F1', '2F2', '3F1', '3F2', '4F1', '4F2'],
    '端口数': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    '小组': ['1', '2', '2', '3', '4', '4', '1', '1', '2', '2', '3', '3', '4', '4'],
    '启动时间': [
        time(8, 0, 0),
        time(8, 1, 0),
        time(8, 1, 0),
        time(8, 2, 0),
        time(8, 3, 0),
        time(8, 3, 0),
        time(8, 0, 0),
        time(8, 0, 0),
        time(8, 1, 0),
        time(8, 1, 0),
        time(8, 2, 0),
        time(8, 2, 0),
        time(8, 3, 0),
        time(8, 3, 0),
        ],
    '采样日程': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # '计划采样点': [147, 147, 147, 147],
}

instrument_df: DataFrame = pd.DataFrame(data=instruments_raw)#.set_index('代号')

instrument_df.head()

In [ ]:
instrument_df_copy = instrument_df.copy()

instrument_df_copy['端口'] = (
    instrument_df_copy['端口数']
    .apply(lambda x: list(range(1, int(x) + 1)))
)
instrument_df_copy['启动时间'] = (
    instrument_df_copy['启动时间']
    .apply(lambda x: datetime.combine(datetime.today(), x))
)
instrument_df_copy = (
    instrument_df_copy
    .assign(
        是否完成=False,
        上一个采样点=0,
        采样次数=0
    )
)

instrument_df_copy.head()

In [ ]:
instrument_df_copy.query('小组 == "1"').set_index('代号').iloc[0].name

In [ ]:
groups: list[int] = instrument_df_copy['小组'].drop_duplicates().tolist()
groups

In [ ]:
gather_types = instrument_df_copy.query('小组 == "1"')['收集方式'].drop_duplicates().tolist()
gather_types

In [ ]:
remainder_rows_query_str: str = f'收集方式 in {gather_types} and 是否完成 == False'
remainder_rows_query_str

In [ ]:
# 当前小组的仪器可采样的点位数量
test_raw_point_df = test_raw_point_df.assign(是否完成 = False)
remainder_df: DataFrame = test_raw_point_df.query(remainder_rows_query_str)
remainder_rows: int = remainder_df.shape[0]

remainder_rows

In [ ]:
# 仪器是否工作结束
if remainder_rows == 0:
    instrument_df_copy.loc[instrument_df_copy['小组'] == 1, '是否完成'] = True
else:
    pass


In [ ]:
days: list[int] = instrument_df_copy['采样日程'].drop_duplicates().tolist()
days

In [ ]:
for day in days:
    for group in groups:
        group_instrument_df: DataFrame = (
            instrument_df_copy
            .query('小组 == @group and 采样日程 == @day')
        )
        print(group_instrument_df, '\r\n')
        boot_time: time = group_instrument_df['启动时间'].max().time()
        # print(boot_time)
        all_time_span: int = 15 + 5
        time_interval: timedelta = timedelta(minutes=all_time_span)
        # print(time_interval)


In [ ]:
break_time_df: DataFrame = pd.DataFrame(data={
    '开始时间': [time(12, 0, 0), time(17, 30, 0)],
    '结束时间': [time(13, 0, 0), time(19, 0, 0)],
})

break_time_df.head()

In [ ]:
sample_project = SampleScheduleManage(test_raw_point_df, instrument_df, break_time_df, time_span=3)

In [ ]:
sample_project.instruments

In [ ]:
sample_project.sample_time_df.head()

In [ ]:
sample_project.work_df.head()

In [ ]:
# sample_project.sample_work()

In [ ]:
# sample_project.work_df.head()

In [ ]:
# round_num: int = int(sample_project.work_df['次序'].max())

# round_num

In [ ]:
# with pd.ExcelWriter('sample_schedule_manage_work_df测试1.xlsx') as writer:
#     sample_project.work_df.to_excel(writer, sheet_name='定点采样', index=False)
#     sample_project.sample_time_df.to_excel(writer, sheet_name='采样次序安排', index=False)

In [ ]:
# sample_project.work_df.to_excel('sample_schedule_manage_work_df测试.xlsx', index=False)

In [ ]:
# value_counts_df = (
#     sample_project
#     .work_df
#     [['采样点编号', '收集方式', '是否完成']]
#     .value_counts()
#     .reset_index()
#     .rename(columns={0: '数量'})
# )

# value_counts_df.head()

In [ ]:
# class GasSampler():
#     """
#     Gas Sampler
#     """
#     def __init__(
#             self,
#             sample_type: str,
#             ports: int,
#             code_name: str,
#             startup_time: time
#         ) -> None:
#         self.code_name: str = code_name
#         self.name: str = '大气采样器'
#         self.sample_type: str = sample_type
#         self.ports: list[int] = list(range(1, ports + 1))
#         self.startup_time: time = startup_time
#         self.last_point: int = 0
#         self.last_sample_time: int = 0
#         self.is_sample: bool = True
    
#     def judge_is_sample(self, df: DataFrame) -> None:
#         '''是否可以采样'''
#         df_sample_count: int = (
#             df
#             .query(f'收集方式 == {self.sample_type} and 是否完成 == False')
#             .shape[0]
#         )
#         if df_sample_count > 0:
#             pass
#         else:
#             self.is_sample = False
    
#     def select_sample_point(self, df: DataFrame):
#         '''选择采样点'''
#         # 所有采样点
#         sample_points: list[int] = df['采样点编号'].drop_duplicates().tolist()
#         # [ ] 判断上一个采样点是否有符合要求的采样点
#         # is_continue_sample: bool = (
#         #     self.is_sample
#         #     and

#         # )
#         if self.is_sample:
#             if self.last_point == 0:
#                 new_point: int = random.choice(sample_points)
#                 self.last_point = new_point
#                 pass
#             pass
    
#     def do_sample(self, df: DataFrame):
#         '''采样'''
#         pass


In [ ]:
# from openpyxl import Workbook
# from openpyxl.worksheet.table import Table, TableStyleInfo

# wb = Workbook()
# ws = wb.active

# data = [
#     ['Apples', 10000, 5000, 8000, 6000],
#     ['Pears',   2000, 3000, 4000, 5000],
#     ['Bananas', 6000, 6000, 6500, 6000],
#     ['Oranges',  500,  300,  200,  700],
# ]

# # add column headings. NB. these must be strings
# ws.append(["Fruit", "2011", "2012", "2013", "2014"])
# for row in data:
#     ws.append(row)

# tab = Table(displayName="Table1", ref="A1:E5")

# # Add a default style with striped rows and banded columns
# style = TableStyleInfo(name="TableStyleMedium10", showFirstColumn=False,
#                        showLastColumn=False, showRowStripes=True, showColumnStripes=True)
# tab.tableStyleInfo = style

# '''
# Table must be added using ws.add_table() method to avoid duplicate names.
# Using this method ensures table name is unque through out defined names and all other table name. 
# '''
# ws.add_table(tab)
# wb.save("table.xlsx")

In [ ]:
# test_df: DataFrame = pd.read_clipboard()

# test_df.head()

In [ ]:
# test_df['点位'] = test_df['车间/单元'].str.cat(
#     [test_df['岗位'], test_df['工种']],
#     sep=' '
#     )

# test_df.head()

In [ ]:
# new_df = test_df[['采样点编号', '点位']].drop_duplicates().reset_index(drop=True)

# new_df.head()

In [ ]:
# address_name_list: list[str] = new_df['点位'].tolist()

In [ ]:
# diff_list: list[list[float]] = []

# for address_i in address_name_list:
#     current_diff_list: list[float] = []
#     for address_j in address_name_list:
#         diff: float = difflib.SequenceMatcher(None, address_i, address_j).ratio()
#         current_diff_list.append(diff)
#     diff_list.append(current_diff_list)
    

In [ ]:
# diff_array = np.array(diff_list)

# diff_array

In [ ]:
# np.savetxt('diff_array.csv', diff_array, delimiter=',')

In [ ]:
# a = '精炼部连铸机 7连铸机钢水下料口（方坯） 中浇工'
# b = '精炼部连铸机 2连铸机钢水下料口（方坯） 中浇工'
# c = '精炼部连铸机 2连铸机钢水下料口(板坯） 中浇工'

In [ ]:
# difflib.SequenceMatcher(None, a, b).ratio()

In [ ]:
# difflib.SequenceMatcher(None, a, c).ratio()

In [ ]:
# difflib.SequenceMatcher(None, b, c).ratio()